In [1]:
try: import simplejson as json
except ImportError: import json

import gzip,codecs,numpy as np,random,copy
import scipy.optimize as opt

In [2]:
with open("ratebeer_train_random.json","r") as infile:
    train = json.load(infile)
infile.close()
with open("ratebeer_test_random.json","r") as infile:
    test = json.load(infile)
infile.close()
with open("ratebeer_quickmap_random.json","r") as infile:
    quickmap = json.load(infile)
infile.close()

print(len(train),len(test),len(quickmap))
train = sorted(train, key = lambda k : k["review/time"])

2704815 4760 2709575


In [5]:
Iu = dict() #set of products reviewed by users
Ui = dict() #set of users who reviewed the product

In [6]:
for review in train:
        item = review["product/productId"]
        user = review["review/userId"]
        if item in Ui:
            Ui[item].append(user)
        else:
            Ui[item] = [user]
        if user in Iu:
            Iu[user].append(item)
        else:
            Iu[user] = [item]

In [7]:
print(len(Iu))

4760


In [8]:
distinct_user_set = set()
distinct_item_set = set()
for review in train:
    if review["review/userId"] not in distinct_user_set:
        distinct_user_set.add(review["review/userId"])
    if review["product/productId"] not in distinct_item_set:
        distinct_item_set.add(review["product/productId"])
print(len(distinct_user_set), len(distinct_item_set))

4760 110032


In [9]:
import sys
sys.setrecursionlimit(20000)

# Expertise modelling

In [72]:
class ExpertiseLFM(object):
    ''' Expertise LFM class implements the evolution latent factor model of collaborative filtering 
    using matrix factorization
    '''
    
    def __init__(self,train_data, Iu_reg, Ui_reg, userproduct_dict,userset,itemset,k,Lambda1,Lambda2,E):
        ''' requires Iu and Ui matrix information, quick mapping of reviews to (user,product),
         k =number of latent factor dimensions, 
         lambda1 = reg parameter, lambda2 = smoothing parameter,
         E = number of experience levels.
        '''
        
        self.Ntrain = len(train_data)               #Number of training samples
        self.train_data = train_data                #training data 
        self.Iu = self.deepish_copy(Iu_reg)         #Iu mapping  
        self.Ui = self.deepish_copy(Ui_reg)         #Ui mapping
        self.quickmap = userproduct_dict            #uses key as (userid-itemid) for quick mapping to required review
        self.user_set = userset
        self.item_set = itemset
        
        self.user_map = {}     #mapping for easier transformation from long gradient vector to individual gradients
        self.item_map = {}
        self.reverse_user_map = {}
        self.reverse_item_map = {}
        i=0
        for user in self.user_set:
            self.user_map[i] = user
            self.reverse_user_map[user] = i
            i+=1
        i=0
        for item in self.item_set:
            self.item_map[i] = item
            self.reverse_item_map[user] = i
            i+=1
        
        #hyperparameters
        self.Lambda1 = Lambda1    #regularization param
        self.Lambda2 = Lambda2    #smoothing reg param     
        self.k = k                # number of latent factor dimension (low dimensional repr)
        self.E = E                #number of experience levels
        
        self.final_param = self.init_theta()  #current final_parameters
        self.init_exp()
    
    
    def init_theta(self):
        ''' Initializes the parameters of E recommender models
        
        flat_theta = <alpha_G, Bu_G, Bi_G, alpha_e1..E, Bu_e1..E, Bi_e1..E, Gu_e1...E, Gi_e1...E>
        '''
        flat_theta = []
        
        rating_arr = [review["review/score"] for review in self.train_data]
        avg_rating = np.mean(rating_arr)
        self.alpha_G = avg_rating                                 #global offset
        self.Bu_G = dict()                                        #user bias (global)
        self.Bi_G = dict()                                        #item bias (global)
        
        for i in range(len(self.user_map)):
            self.Bu_G[self.user_map[i]] = np.random.random(1).item() 
        for i in range(len(self.item_map)):
            self.Bi_G[self.item_map[i]] = np.random.random(1).item() 
        
        flat_theta.append(self.alpha_G)
        flat_theta.extend(list(self.Bu_G.values()))
        flat_theta.extend(list(self.Bi_G.values()))
        
        self.alpha = np.random.rand(self.E)                     #individual offset parameters per exp
        self.Bu = [dict() for i in range(self.E)]               #user bias per exp
        self.Bi = [dict() for i in range(self.E)]               #item bias per exp
        self.Gu = [dict() for i in range(self.E)]               #user latent factor vector repr per exp
        self.Gi = [dict() for i in range(self.E)]               #item latent factor vector repr per exp
        
        flat_theta.extend(self.alpha)
        for e in range(self.E):
            for i in range(len(self.user_map)):
                self.Bu[e][self.user_map[i]] = np.random.random(1).item()
                flat_theta.append(self.Bu[e][self.user_map[i]]) 
        for e in range(self.E):        
            for j in range(len(self.item_map)):
                self.Bi[e][self.item_map[j]] = np.random.random(1).item() 
                flat_theta.append(self.Bi[e][self.item_map[j]])
        for e in range(self.E):
            for i in range(len(self.user_map)):
                self.Gu[e][self.user_map[i]] = np.random.uniform(0,1,(1,self.k))
                flat_theta.extend(np.array(list(self.Gu[e][self.user_map[i]])).flatten())
        for e in range(self.E):        
            for j in range(len(self.item_map)):
                self.Gi[e][self.item_map[j]] = np.random.uniform(0,1,(1,self.k))
                flat_theta.extend(np.array(list(self.Gi[e][self.item_map[j]])).flatten())
        
        self.recparam = (1 + len(self.user_set) + len(self.item_set) \
                            + self.k*(len(self.user_set) + len(self.item_set))) #per experience level parameters
        self.globalparam = 1 + len(self.user_set) + len(self.item_set)   #global parameters
        self.totalparams = self.recparam * self.E + self.globalparam
        
        return np.array(flat_theta)
    
    def init_exp(self):
        ''' Initializes experience for each user-item combination uniformly over rating time'''
        self.eui = dict()  #experience dictionary of dictionaries (1: user level, 2: item level)
        for user in self.user_set:
            self.eui[user] = {}
            num_items_in_level = round(len(self.Iu[user])/self.E)
            if num_items_in_level ==0:
                print(len(self.Iu[user]),user)
            cur_level = 0
            item = self.Iu[user][0]
            self.eui[user][item] = cur_level
            for i in range(1,len(self.Iu[user])):
                if i% num_items_in_level == 0 and cur_level != self.E-1 :
                     cur_level+=1
                item = self.Iu[user][i]
                self.eui[user][item] = cur_level
                
    def OPT_rec(self,i,j,n,user):
        '''
         i = current experience level, j = jth rating of user, n = number of ratings given by user
         internally modelled as experience 0 to E-1 (so experience E is invalid)
        '''
        if i==self.E or j==n:
            return np.inf
        elif self.OPT[i,j] >=0: #intial value = -1
            return self.OPT[i,j]
        else:
            item = self.Iu[user][j]  # jth rating
            rating = self.quickmap[user+"-"+item]["review/score"]
            temp  = min(self.OPT_rec(i+1,j+1,n,user), self.OPT_rec(i,j+1,n,user))
            rec_e = self.alpha_G + self.Bu_G[user] + self.Bi_G[item] + \
                    self.alpha[i] + self.Bu[i][user] + self.Bi[i][item] + \
                    np.asscalar(np.dot(self.Gu[i][user], self.Gi[i][item].T))
            if temp == np.inf:
                self.OPT[i,j] = (rec_e - rating)**2 
            else:
                self.OPT[i,j] = (rec_e - rating)**2 + temp
                  
            return self.OPT[i,j]
        
    def assign_exp_level(self):
        ''' Using the DP solution similar to Longest Common SubSequence, predict new experience level
        for each user-item combination'''
        k = 0
        count=0
        for user in self.Iu:
            n = len(self.Iu[user])
            self.OPT = np.matrix([[-1.0]*n]*self.E)  #initialize to invalid values
            for i in range(self.E):
                self.OPT_rec(i,0,n,user)
            cur_level = np.argmin(self.OPT[:,0])
            j = 0
            item = self.Iu[user][j]
            self.eui[user][item] = cur_level
            start_level = cur_level
            j+=1
            while (j < n):
                try: 
                    if cur_level != self.E-1 and self.OPT[cur_level,j] >= self.OPT[cur_level+1,j]:
                        cur_level +=1
                    item = self.Iu[user][j]
                    if cur_level != self.eui[user][item]:
                        count+=1
                    self.eui[user][item] = cur_level
                    j+=1
                except Exception as e:
                    print(e.args,i,j,n)
            if k%1000==0:
                print("user: {} start level: {} end level: {}".format(user,start_level,cur_level))
            k+=1 
        print("Number of experience levels changed:{}".format(count))
        return count

    def retrieve_theta_components(self,theta):
        ''' Sets all parameters from the long theta vector obtained after update rule'''
        j = 0
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        self.alpha_G = theta[j]
        j+=1
        for i in range(umap_len):
            self.Bu_G[self.user_map[i]] = theta[j]
            j+=1
        for i in range(imap_len):
            self.Bi_G[self.item_map[i]] = theta[j]
            j+=1 
        for e in range(self.E):
            self.alpha[e] = theta[j]
            j+=1   
        for e in range(self.E):
            for i in range(umap_len):
                self.Bu[e][self.user_map[i]] = theta[j]
                j+=1
        for e in range(self.E):
            for i in range(imap_len):
                self.Bi[e][self.item_map[i]] = theta[j]
                j+=1 
        for e in range(self.E):
            for i in range(umap_len):
                self.Gu[e][self.user_map[i]] = np.array(theta[j:j+self.k])
                j+=self.k  
        for e in range(self.E):
            for i in range(imap_len):
                self.Gi[e][self.item_map[i]] = np.array(theta[j:j+self.k])
                j+=self.k                    
    
    def pred_e(self,user,item,e):
        return self.alpha_G + self.Bu_G[user] + self.Bi_G[item] +\
                      self.alpha[e] + self.Bu[e][user] + self.Bi[e][item] +\
                      np.asscalar(np.dot(self.Gu[e][user], self.Gi[e][item].T))
    
    def f(self,theta):
        '''Calculates the value of the objective function (loss) on the training data. '''
        self.retrieve_theta_components(theta)
        #mean squared error
        error = 0
        for review in self.train_data:
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            error += (self.pred_e(user,item,e) - review["review/score"])**2
        error /= self.Ntrain
        #regularization terms
        reg_complexity = 0
        #ignore global values for now in regularization
        #Bu_np = np.array(list(self.Bu_G.values()))
        #Bi_np = np.array(list(self.Bi_G.values()))
        #reg_complexity = np.sum(np.square(Bu_np)) + np.sum(np.square(Bi_np))
        for e in range(self.E):
            reg_complexity += np.square(self.alpha[e])
            Bu_np = np.array(list(self.Bu[e].values()))
            Bi_np = np.array(list(self.Bi[e].values()))
            reg_complexity += np.sum(np.square(Bu_np)) + np.sum(np.square(Bi_np))
            for user in self.Gu[e]:
                reg_complexity += np.linalg.norm(self.Gu[e][user])**2
            for item in self.Gi[e]:
                reg_complexity += np.linalg.norm(self.Gi[e][item])**2
        #regularization (smoothing cost)
        reg_term = 0
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        for e in range(1,self.E):
            reg_term += (self.alpha[e-1] - self.alpha[e])**2
        for e in range(1,self.E):
            for i in range(umap_len):
                reg_term += (self.Bu[e-1][self.user_map[i]] - self.Bu[e][self.user_map[i]])**2
        for e in range(self.E):
            for i in range(imap_len):
                reg_term += (self.Bi[e-1][self.item_map[i]] - self.Bi[e][self.item_map[i]])**2       
        for e in range(self.E):
            for i in range(umap_len):
                reg_term += np.linalg.norm(self.Gu[e-1][self.user_map[i]] - self.Gu[e][self.user_map[i]])**2  
        for e in range(self.E):
            for i in range(imap_len):
                reg_term += np.linalg.norm(self.Gi[e-1][self.item_map[i]] - self.Gi[e][self.item_map[i]])**2
        return (error + self.Lambda1* reg_complexity +  self.Lambda2 * reg_term)*0.5
    
    def fprime(self, theta):
        ''' Calculates the gradient of objective function f()'''
        self.retrieve_theta_components(theta)
        flat_gradient = []
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        #compute gradient wrt global parameters
        flat_gradient.append(self.compute_gradient_wrt_alpha_global())
        Bu_grad = self.compute_gradient_wrt_Bu_global()
        Bi_grad = self.compute_gradient_wrt_Bi_global()
        flat_gradient.extend(list(Bu_grad.values()))
        flat_gradient.extend(list(Bi_grad.values()))
        #compute gradient wrt experience parameters
        for e in range(self.E):
            flat_gradient.append(self.compute_gradient_wrt_alpha(e))
        for e in range(self.E): 
            Bu_grad = self.compute_gradient_wrt_Bu(e)
            flat_gradient.extend(list(Bu_grad.values()))
        for e in range(self.E):
            Bi_grad = self.compute_gradient_wrt_Bi(e)
            flat_gradient.extend(list(Bi_grad.values()))
        for e in range(self.E):
            Gu_grad = self.compute_gradient_wrt_Gu(e)
            flat_gradient.extend(np.array(list(Gu_grad.values())).flatten())
        for e in range(self.E):
            Gi_grad = self.compute_gradient_wrt_Gi(e)
            flat_gradient.extend(np.array(list(Gi_grad.values())).flatten())
        return np.array(flat_gradient)
    
    def vanilla_gd(self,eta,guess):
        self.i =0;
        flat_theta_guess = guess
        for i in range(100):
            flat_gradient = self.fprime(flat_theta_guess)
            flat_theta_guess -= eta*flat_gradient
            if i%50 ==0: print("{} U : Objective value: {}".format(i,self.f(flat_theta_guess)))
            self.i +=1
        return flat_theta_guess
    
    def vanilla_als (self,eta):
        guess = self.init_theta()
        #print("param = ",guess[self.recparam])
        for m in range(100):
            guess = self.vanilla_gd(eta, guess)
            self.final_param = guess
            self.retrieve_theta_components(guess)
            count = self.assign_exp_level()
            if count==0:
                print("Breaking")
                return

    def call(self,theta):
        print("{} Objective value: {}".format(self.i, self.f(theta)))
        self.i+=1
    
    def objectiveloss_lbfgs(self,thetaguess, grad_tolerance):
        self.i =0;
        flat_theta_guess = thetaguess
        flat_theta,value,d = opt.fmin_l_bfgs_b(self.f,flat_theta_guess,self.fprime,\
                                              pgtol = grad_tolerance,disp=True,\
                                              maxiter = 10, callback = self.call, iprint=0)
        #set the final parameters to the final value returned by fmin_l_bfgs_b
        return flat_theta
    
    
    def push_model(self):
        ''' push the model towards more regularized place'''
        e_alpha_avg = np.mean(self.alpha)
        self.alpha_G += e_alpha_avg
        self.alpha -= e_alpha_avg
        
        for user in self.Bu_G:
            e_Bu_avg = 0 
            for e in range(self.E):
                e_Bu_avg += self.Bu[e][user]
            e_Bu_avg /= self.E
            self.Bu_G[user] += e_Bu_avg
            for e in range(self.E):
                self.Bu[e][user] -= e_Bu_avg
        for item in self.Bi_G:
            e_Bi_avg = 0
            for e in range(self.E):
                e_Bi_avg += self.Bi[e][item]
            e_Bi_avg /= self.E
            self.Bi_G[item] += e_Bi_avg
            for e in range(self.E):
                self.Bi[e][item] -= e_Bi_avg
    
    def als (self,grad_tolerance):
        ''' bad name. not exactly ALS, but performs LBFGS gradient descent, and sets experience level'''
        guess = self.final_param
        for m in range(10):
            print("Iteration {}:".format(m+1))
            print("Objective function value: {}".format(self.f(guess)))
            guess = self.objectiveloss_lbfgs(guess, grad_tolerance)
            self.final_param = guess.copy()
            self.retrieve_theta_components(guess)
            self.push_model()
            print("Model alpha parameters: ",[a + self.alpha_G for a in self.alpha])
            count = self.assign_exp_level()
            print("Objective function value: {}".format(self.f(guess)))
            if count==0:
                print("Breaking")
                return
            

    def mse_test(self,test_data):
        ''' Uses Mean Squared Error as evaluation metric on test data provided by user'''
        self.retrieve_theta_components(self.final_param)
        error = 0
        unknown_data_count =0;
        for review in test_data:
            user = review["review/userId"]
            item = review["product/productId"]
            #assign nearest experience to user-item combo
            rtime = review["review/time"]
            time_arr = []
            for it in self.Iu[user]:
                time_arr.append(self.quickmap[user+"-"+it]["review/time"])
            
            if all(time_arr[i] <= time_arr[i+1] for i in range(len(time_arr)-1))=="False":
                print("raising error. Something went wrong. List should be sorted by default")
            index = np.searchsorted(time_arr,rtime)
            if index == len(self.Iu[user]):
                closest_it = self.Iu[user][index-1]
            else:
                closest_it = self.Iu[user][index]
            
            e = self.eui[user][closest_it]
            
            try:
                error += (self.pred_e(user,item,e) - review["review/score"])**2
            except Exception as e:
                print(e)
                unknown_data_count+=1
        
        if unknown_data_count>0:
            print("Warning! Unknown {} new data rows; Incorporating this into MSE".format(unknown_data_count))
        return error / (len(test_data) - unknown_data_count)
    
    def compute_gradient_wrt_alpha_global(self):
        tempsum = 0
        for review in self.train_data:  #each user item id combo
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            tempsum += ( self.pred_e(user,item,e) - review["review/score"])
        tempsum /= self.Ntrain
        return tempsum
        
    def compute_gradient_wrt_Bu_global(self):
        Bu_grad = {}
        for user in self.Bu_G: 
            total = 0.0
            for item in self.Iu[user]:
                e = self.eui[user][item]
                total += (self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            #total += self.Lambda1*self.Bu_G[user]
            Bu_grad[user] = total
        return Bu_grad
    
    def compute_gradient_wrt_Bi_global(self):
        Bi_grad = {}
        for item in self.Bi_G:
            total = 0.0
            for user in self.Ui[item]:
                e = self.eui[user][item]
                total += (self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            #total += self.Lambda1*self.Bi_G[item]
            Bi_grad[item] = total
        return Bi_grad
    
    def compute_gradient_wrt_alpha(self,exp):
        ''' Compute gradient of objective with respect to alpha parameter of given experience exp level'''
        tempsum = 0
        for review in self.train_data:  #each user item id combo
            user = review['review/userId']
            item = review["product/productId"]
            e = self.eui[user][item]
            if e == exp:  #only take the values pertaining the current level
                tempsum += (self.pred_e(user,item,e) - review["review/score"])
        
        tempsum /= self.Ntrain
        
        #regularization term
        tempsum += self.Lambda1*self.alpha[exp]
        
        if exp == self.E-1:
            tempsum += self.Lambda2 * (self.alpha[exp] - self.alpha[exp-1])
        elif exp == 0:
            tempsum += self.Lambda2 * (self.alpha[exp] - self.alpha[exp+1])
        else:
            tempsum += self.Lambda2 * (2*self.alpha[exp] - self.alpha[exp-1]  - self.alpha[exp+1])
        return tempsum
    
    def compute_gradient_wrt_Bu(self,e):
        ''' Compute gradient of objective with respect to Bu parameter'''
        Bu_grad = {}
        for user in self.Bu[e]: 
            total = 0.0
            for item in self.Iu[user]:
                if self.eui[user][item] == e:
                    total += (self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bu[e][user]
            if e == self.E-1:
                total += self.Lambda2* (self.Bu[e][user] - self.Bu[e-1][user])
            elif e==0:
                total += self.Lambda2* (self.Bu[e][user] - self.Bu[e+1][user])
            else:
                total += self.Lambda2* (2*self.Bu[e][user] - self.Bu[e-1][user]  - self.Bu[e+1][user])
            Bu_grad[user] = total
        return Bu_grad
    
    def compute_gradient_wrt_Bi(self,e):
        ''' Compute gradient of objective with respect to Bi parameter'''
        Bi_grad = {}
        for item in self.Bi[e]:
            total = 0.0
            for user in self.Ui[item]:
                if self.eui[user][item] == e:
                    total +=  (self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"]) 
            total /= self.Ntrain
            total += self.Lambda1*self.Bi[e][item]
            if e == self.E-1:
                total += self.Lambda2* (self.Bi[e][item] - self.Bi[e-1][item])
            elif e==0:
                total += self.Lambda2* (self.Bi[e][item] - self.Bi[e+1][item])
            else:
                total += self.Lambda2* (2*self.Bi[e][item] - self.Bi[e-1][item]  - self.Bi[e+1][item])
            Bi_grad[item] = total
        return Bi_grad
    
    def compute_gradient_wrt_Gu(self,e):
        ''' Compute gradient of objective with respect to Gu parameter'''
        Gu_grad  = {} 
        for user in self.Gu[e]:   
            total = np.zeros((1,self.k)) 
            for item in self.Iu[user]:
                if self.eui[user][item] == e:
                    total+= np.multiply(self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"],\
                            self.Gi[e][item])
            total /= self.Ntrain
            total += self.Lambda1*self.Gu[e][user]
            if e == self.E-1:
                total += self.Lambda2* (self.Gu[e][user] - self.Gu[e-1][user]) 
            elif e==0:
                total += self.Lambda2* (self.Gu[e][user] - self.Gu[e+1][user]) 
            else:
                total += self.Lambda2* (2*self.Gu[e][user] - self.Gu[e-1][user]  - self.Gu[e+1][user])
            Gu_grad[user] = total.copy()
        return Gu_grad
    
    def compute_gradient_wrt_Gi(self,e):
        ''' Compute gradient of objective with respect to Gi parameter'''
        Gi_grad = {}
        for item in self.Gi[e]:
            total = np.zeros((1,self.k))
            for user in self.Ui[item]:
                if self.eui[user][item] == e:

                    total+= np.multiply(self.pred_e(user,item,e) - self.quickmap[user+'-'+item]["review/score"],\
                                self.Gu[e][user])
            total /= self.Ntrain
            total += self.Lambda1*self.Gi[e][item]
            if e == self.E-1:
                total += self.Lambda2* (self.Gi[e][item] - self.Gi[e-1][item])
            elif e==0:
                total += self.Lambda2* (self.Gi[e][item] - self.Gi[e][item]) 
            else:
                total += self.Lambda2* (2*self.Gi[e][item] - self.Gi[e-1][item]  - self.Gi[e+1][item])
            Gi_grad[item] = total.copy()
        return Gi_grad
    
        
    def deepish_copy(self,org):
        '''much, much faster than deepcopy, for a dict of the simple python types.'''
        out = dict().fromkeys(org)
        for k,v in org.items():
            try:
                out[k] = v.copy()   # dicts, sets
            except AttributeError:
                try:
                    out[k] = v[:]   # lists, tuples, strings, unicode
                except TypeError:
                    out[k] = v      # ints

        return out  

In [ ]:
lfmObj = ExpertiseLFM(train,Iu, Ui,quickmap,distinct_user_set,distinct_item_set,5,1,1,5)

In [ ]:
lfmObj.als(1e-2)

Iteration 1:
Objective function value: 860568.1384148761
0 Objective value: 710272.2823469071
1 Objective value: 106161.61723093552
2 Objective value: 24991.502673103772
3 Objective value: 1883.2286103063877
4 Objective value: 660.7436159156289
5 Objective value: 8.738692503028675
6 Objective value: 2.990815129697837
7 Objective value: 1.0963730616452478
8 Objective value: 0.4179673448355708
9 Objective value: 0.40978073456530195


In [15]:
lfmObj.als(1e-2)

Iteration 1:
Objective function value: 880208.1080163699
0 Objective value: 728833.9027478835
1 Objective value: 108207.58523556212
2 Objective value: 24787.280796043397
3 Objective value: 1950.2537881528115
4 Objective value: 672.119160097023
5 Objective value: 9.154300848019473
6 Objective value: 3.1066645595098357
7 Objective value: 1.0330595149330162
8 Objective value: 0.3473650732783926
9 Objective value: 0.3329269557337709
10 Objective value: 0.32670700228618477
11 Objective value: 0.32629803332274704
12 Objective value: 0.32598298158129635
[3.2888581820523148, 3.2908307625414177, 3.2933762948906051, 3.298336639988237, 3.3017812421983144]
user: VA Homebrewer start level: 0 end level: 4
user: LagerLove start level: 1 end level: 4
user: JJClark start level: 2 end level: 4
user: TikiE start level: 0 end level: 4
user: rakkasan start level: 0 end level: 0
Number of experience levels changed:2012023
Objective function value: 0.3244345553669969
Iteration 2:
Objective function value: 0.

In [46]:
lfmObj.mse_test(test)

'18186'
'27384'
'33304'
'34717'
'54832'
'65106'
'84699'
'108813'
'114376'
'119595'
'120160'
'124485'
'156825'
Warning! Unknown 13 new data rows


15.62359128935382